# Implicit Trajectory combined with RRT #
- Work space: 3D with obstacles
- Obstacle type: static and known
- Agent Model: Dubins airplane (Owen2015)
- Integration Algorithm: 4th Order Runge Kutta
- Implicit Curve: Time-invariant Circle
- Distance Metric: admissionable heuristic, based on time-optimal dubins-airplane paths (altitude difference is integrated in a linear fashion)

In [2]:
# load the code
%run 'MAIN_CODE.ipynb'
import random
random.seed( 99 ) #arbitrary

# Performance Experiments

## 1st stage: VF-RRT+impl.Curve vs. RRT+goal bias

In [3]:
import time

#Parameters in General
NumOfExperiments = 100
qstart = np.array([5.,5.,2.,np.pi])# specific initial point
MaxNodes = 200
MAXiter = MaxNodes*3

# Experiment parameters for VF-RRT
lambdaNsteps=20
Es=0.85
lamdb=1.0

# Experiment parameters for naive RRT
goalBias=0.35

In [4]:
#specific obstacles, fixed, known, not moving
# reset the RNG
random.seed( 500 ) #arbitrary
Obstacles = CreateSphericalObstacles(N=100,r=3.)
while isBlocked(qstart, Obstacles):
    Obstacles = CreateSphericalObstacles(N=100,r=3.)

In [5]:
# initilize
VFRRTnodes = list()
VFRRTupstream = list()
VFRRTnodes = list()
VFRRTexetime = list()
VFRRTwork = list()
VFRRTsuccess = list()
VFRRTiter = list()
VFRRTeuclLength = list()
VFRRTanglLength = list()
    
RRTnodes = list()
RRTupstream = list()
RRTnodes = list()
RRTexetime = list()
RRTwork = list()
RRTsuccess = list()
RRTiter = list()
RRTeuclLength = list()
RRTanglLength = list()

random.seed( 500 ) #arbitrary
print('VFRRT')
print('#\tt\tsuccess\twork\tupstr\tnodes\titer\teuDist\tangDist\thiLim\tloLim')
for IofExp in range(0,NumOfExperiments):
    # Execute and return Tree
    stampBegin = time.process_time()
    T = VFRRT(qstart,VF,lambdaNsteps,Es,lamdb,MaxNodes,MAXiter,Obstacles,bounds=bounds)
    CPUtimePassed = time.process_time()-stampBegin

    
    if T.successful:
        VFRRTexetime.append(CPUtimePassed)
        VFRRTiter.append(T.iterations)
        VFRRTsuccess.append(IofExp)
        IDpath,Datapath,Edgepath = T.get_solution()
        work,upstream = T.get_WorkAndUpstreamOfPath(Datapath)
        euclDist,anglDist = T.get_length(Datapath)
        VFRRTnodes.append(T.NumNodes)
        VFRRTupstream.append(upstream)
        VFRRTwork.append(work)
        VFRRTeuclLength.append(euclDist)
        VFRRTanglLength.append(anglDist)
        
        print('%d\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\t%s\t%s' % (IofExp,CPUtimePassed,work,upstream,T.NumNodes,T.iterations,euclDist,anglDist,str(T.lambd_high),str(T.lambd_low)))
    else:
        print('%d\t%.3f\tFalse\t\t\t%d\t%d\t\t\t%s\t%s' % (IofExp,CPUtimePassed,T.NumNodes,T.iterations,str(T.lambd_high),str(T.lambd_low)))
    
    #T.plot(DrawTrajectory=True,DrawCollisionNodes=True)
    #T.plot(obs=Obstacles,OnlyClosestObs=True,DrawCollisionNodes=True)
    #DrawStats(T)

print('RRT')
bestqgoal = np.array([0.,0.,0.,0.])
bestDist = np.inf
for IofExp in range(0,100):
    gt = np.mod(IofExp/NumOfExperiments*2*np.pi+np.pi/2,2*np.pi)
    qgoal=(
        xc+r*np.cos(2*np.pi*IofExp/NumOfExperiments),
        yc+r*np.sin(2*np.pi*IofExp/NumOfExperiments),
        zc,
        gt
    )
    tempDist = dubinsDist(qstart,qgoal)
    if tempDist<bestDist:
        bestqgoal = qgoal
        bestDist = tempDist
print("Best qgoal for RRT: [%.2f,%.2f,%.2f,%.2f] with dist %.2f" % (bestqgoal[0],bestqgoal[1],bestqgoal[2],bestqgoal[3],bestDist))
print('#\tt\tsuccess\twork\tupstr\tnodes\titer\teuDist\tangDist')

for IofExp in range(0,NumOfExperiments):
    stampBegin = time.process_time()
    #pdb.set_trace()
    T = RRT(qstart,qgoal,MaxNodes,MAXiter,Obstacles,bounds,goalBias)
    CPUtimePassed = time.process_time()-stampBegin

    
    if T.successful:
        RRTexetime.append(CPUtimePassed)
        RRTiter.append(T.iterations)
        T.VF = VF
        RRTsuccess.append(IofExp)
        IDpath,Datapath,Edgepath = T.get_solution()
        work,upstream = T.get_WorkAndUpstreamOfPath(Datapath)
        euclDist,anglDist = T.get_length(Datapath)
        RRTnodes.append(T.NumNodes)
        RRTupstream.append(upstream)
        RRTwork.append(work)
        RRTeuclLength.append(euclDist)
        RRTanglLength.append(anglDist)
        
        print('%d\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f' % (IofExp,CPUtimePassed,work,upstream,T.NumNodes,T.iterations,euclDist,anglDist))
    else:
        print('%d\t%.3f\tFalse\t\t\t%d\t%d\t\t' % (IofExp,CPUtimePassed,T.NumNodes,T.iterations))
    #T.plot(obs=Obstacles,OnlyClosestObs=True,DrawCollisionNodes=True)
#plt.show()





VFRRT
#	t	success	work	upstr	nodes	iter	euDist	angDist	hiLim	loLim
0	0.933	True	4.608	2.384	121	153	4.862	10.567	False	False
1	0.316	True	4.602	1.051	67	67	4.714	13.190	False	False
2	0.348	True	4.521	1.737	75	75	4.708	11.880	False	False
3	0.333	True	4.586	1.090	75	75	4.704	12.221	False	False
4	1.927	False			200	354			False	False
5	0.338	True	4.565	2.128	72	72	4.791	13.519	False	False
6	0.274	True	4.550	1.613	61	61	4.729	12.116	False	False
7	0.361	True	4.647	1.541	79	79	4.811	9.961	False	False
8	0.381	True	4.546	1.626	74	74	4.719	12.264	False	False
9	0.343	True	4.676	2.502	75	75	4.948	13.610	False	False
10	0.284	True	4.676	1.137	67	67	4.798	11.868	False	False
11	1.953	False			200	365			False	False
12	0.299	True	4.642	1.561	70	70	4.808	11.847	False	False
13	1.037	True	4.644	1.470	153	200	4.804	12.239	False	False
14	0.436	True	4.658	1.182	84	103	4.785	12.716	False	False
15	0.373	True	4.585	1.305	82	82	4.724	12.185	False	False
16	0.361	True	4.560	1.248	82	82	4.698	10.283	False	False
17	0.3

## Boxplot of the Data

In [6]:
#VFRRT
print('VFRRT')
print('\nStats for successful runs only\n%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmin' % (
                                                                len(VFRRTsuccess)*100./NumOfExperiments,
                                                                np.min(VFRRTexetime),
                                                                np.min(VFRRTwork),
                                                                np.min(VFRRTupstream),
                                                                np.min(VFRRTnodes),
                                                                np.min(VFRRTiter),
                                                                np.min(VFRRTeuclLength),
                                                                np.min(VFRRTanglLength)))
print('%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmean' % (
                                                                len(VFRRTsuccess)*100./NumOfExperiments,
                                                                np.mean(VFRRTexetime),
                                                                np.mean(VFRRTwork),
                                                                np.mean(VFRRTupstream),
                                                                np.mean(VFRRTnodes),
                                                                np.mean(VFRRTiter),
                                                                np.mean(VFRRTeuclLength),
                                                                np.mean(VFRRTanglLength)))
print('%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmax' % (
                                                                len(VFRRTsuccess)*100./NumOfExperiments,
                                                                np.max(VFRRTexetime),
                                                                np.max(VFRRTwork),
                                                                np.max(VFRRTupstream),
                                                                np.max(VFRRTnodes),
                                                                np.max(VFRRTiter),
                                                                np.max(VFRRTeuclLength),
                                                                np.max(VFRRTanglLength)))
print('\n%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tstd\n' % (
                                                                len(VFRRTsuccess)*100./NumOfExperiments,
                                                                np.std(VFRRTexetime),
                                                                np.std(VFRRTwork),
                                                                np.std(VFRRTupstream),
                                                                np.std(VFRRTnodes),
                                                                np.std(VFRRTiter),
                                                                np.std(VFRRTeuclLength),
                                                                np.std(VFRRTanglLength)))

if len(RRTsuccess):
    #RRT
    print('\nStats for successful runs only\n%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmin' % (
                                                                    len(RRTsuccess)*100./NumOfExperiments,
                                                                    np.min(RRTexetime),
                                                                    np.min(RRTwork),
                                                                    np.min(RRTupstream),
                                                                    np.min(RRTnodes),
                                                                    np.min(RRTiter),
                                                                    np.min(RRTeuclLength),
                                                                    np.min(RRTanglLength)))

    print('%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmean' % (
                                                                    len(RRTsuccess)*100./NumOfExperiments,
                                                                    np.mean(RRTexetime),
                                                                    np.mean(RRTwork),
                                                                    np.mean(RRTupstream),
                                                                    np.mean(RRTnodes),
                                                                    np.mean(RRTiter),
                                                                    np.mean(RRTeuclLength),
                                                                    np.mean(RRTanglLength)))
    print('%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tmax' % (
                                                                    len(RRTsuccess)*100./NumOfExperiments,
                                                                    np.max(RRTexetime),
                                                                    np.max(RRTwork),
                                                                    np.max(RRTupstream),
                                                                    np.max(RRTnodes),
                                                                    np.max(RRTiter),
                                                                    np.max(RRTeuclLength),
                                                                    np.max(RRTanglLength)))
    print('\n%.1d%%\t%.3f\tTrue\t%.3f\t%.3f\t%d\t%d\t%.3f\t%.3f\tstd\n' % (
                                                                    len(RRTsuccess)*100./NumOfExperiments,
                                                                    np.std(RRTexetime),
                                                                    np.std(RRTwork),
                                                                    np.std(RRTupstream),
                                                                    np.std(RRTnodes),
                                                                    np.std(RRTiter),
                                                                    np.std(RRTeuclLength),
                                                                    np.std(RRTanglLength)))
    
    # Both for Latex
    print(  '\\begin{table}\n'
            '\\centering\n'
            '\\caption{Stage1: Comparison of biased native RRT and proposed Algorithm \\ref{alg:stage2} to find a path to the curve}\n'
            '\\label{tab:results-stage1-icvfrrt}\n'
            '\\begin{tabular}{|c|c|c|}\n'
            '	\\hline & \\specialcell{Proposed\\\\icVFRRT} & \\specialcell{Native RRT\\\\with Goal Bias}  \\\\ \n'
            '	\\hline Successful Runs & %.1d\\%% & %.1d\\%% \\\\ \n'
            '	\\hline Time in s & %.2f & %.2f \\\\ \n'
            '	\\hline Nodes in $\\varGamma$ & %.2f & %.2f \\\\ \n'
            '	\\hline Iterations & %.2f & %.2f \\\\ \n'
            '	\\hline Solution Length in m & %.2f & %.2f \\\\ \n'
            '	\\hline Solution Rotations in rad & %.2f & %.2f \\\\ \n'
            '	\\hline Solution upstream  & %.2f & %.2f \\\\ \n'
            '	\\hline \n'
            '\\end{tabular} \n'
            '\\end{table}'%(
            len(VFRRTsuccess)*100./NumOfExperiments,len(RRTsuccess)*100./NumOfExperiments,
            np.mean(VFRRTexetime),np.mean(RRTexetime),
            np.mean(VFRRTnodes),np.mean(RRTnodes),
            np.mean(VFRRTiter),np.mean(RRTiter),
            np.mean(VFRRTeuclLength),np.mean(RRTeuclLength),
            np.mean(VFRRTanglLength),np.mean(RRTanglLength),
            np.max(VFRRTupstream),np.max(RRTupstream)
            )
         )


labels = list(['icVFRRT','RRT'])
fs = 10  # fontsize

# demonstrate how to toggle the display of different elements:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(6, 6))

axes[0, 0].boxplot([VFRRTeuclLength,RRTeuclLength], labels=labels)
axes[0, 0].set_title('Path Length', fontsize=fs)

axes[0, 1].boxplot([VFRRTexetime,RRTexetime], labels=labels)
axes[0, 1].set_title('Time', fontsize=fs)

axes[1, 0].boxplot([VFRRTnodes,RRTnodes], labels=labels)
axes[1, 0].set_title('Tree Nodes', fontsize=fs)

axes[1, 1].boxplot([VFRRTanglLength,RRTanglLength], labels=labels)
axes[1, 1].set_title('Cumulative Angular Differences', fontsize=fs)

plt.show()

VFRRT

Stats for successful runs only
96%	0.274	True	4.454	0.747	59	59	4.597	8.073	min
96%	0.465	True	4.603	1.704	85	96	4.787	12.481	mean
96%	1.915	True	4.795	5.000	194	349	5.040	16.198	max

96%	0.287	True	0.072	0.726	25	53	0.099	1.727	std


Stats for successful runs only
19%	0.284	True	2.791	31.037	96	95	6.568	19.982	min
19%	0.453	True	3.228	35.638	117	116	7.013	23.763	mean
19%	1.113	True	3.504	41.723	166	165	7.765	27.972	max

19%	0.185	True	0.141	2.931	19	19	0.301	2.416	std

\begin{table}
\centering
\caption{Stage1: Comparison of biased native RRT and proposed Algorithm \ref{alg:stage2} to find a path to the curve}
\label{tab:results-stage1-icvfrrt}
\begin{tabular}{|c|c|c|}
	\hline & \specialcell{Proposed\\icVFRRT} & \specialcell{Native RRT\\with Goal Bias}  \\ 
	\hline Successful Runs & 96\% & 19\% \\ 
	\hline Time in s & 0.46 & 0.45 \\ 
	\hline Nodes in $\varGamma$ & 85.22 & 117.32 \\ 
	\hline Iterations & 96.79 & 116.53 \\ 
	\hline Solution Length in m & 4.79 & 7.01 \\ 
	\hline So